In [1]:
model_name = "pos_cash_balance"

In [2]:
import numpy as np
import pandas as pd
import gc
from time import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [3]:
timesheet = [time()]
def timer(statement):
    global timesheet
    timesheet.append(time())
    print statement+" :", (timesheet[-1]-timesheet[-2]),"seconds"

# Loading Data

In [4]:
pos_cash_balance = pd.read_csv("../extractor/csv/pos_cash_features_normalized_V1.csv").dropna(axis=1, how="all")

train_id = pd.read_csv("../data/application_train.csv", usecols = ["SK_ID_CURR","TARGET"])
test_id = pd.read_csv("../data/application_test.csv", usecols = ["SK_ID_CURR"])

pos_cash_balance = pos_cash_balance.merge(train_id, how = "left", on = "SK_ID_CURR")

train = pos_cash_balance.loc[pos_cash_balance.SK_ID_CURR.isin(train_id.SK_ID_CURR)].reset_index(drop=True)
test = pos_cash_balance.loc[pos_cash_balance.SK_ID_CURR.isin(test_id.SK_ID_CURR)].reset_index(drop=True)

partial_train_id = train[["SK_ID_CURR"]]
partial_test_id = test[["SK_ID_CURR"]]
target =train.TARGET

train.drop(["SK_ID_CURR", "TARGET"], axis=1, inplace=True)
test.drop(["SK_ID_CURR", "TARGET"], axis=1, inplace = True)

# Defining Model

In [5]:
from sklearn.linear_model import LogisticRegression
def model_lr(x_train, x_test, y_train, y_test, test, meta_train, meta_test,train_index, test_index,fold_id):
    clf = LogisticRegression(class_weight = {1: np.mean(target), 0: (1-np.mean(target))}, random_state = 123)
    clf.fit(x_train, y_train)
    
    meta_train[test_index] = clf.predict_proba(x_test)[:,1]
    meta_test.append(clf.predict_proba(test)[:,1])

    print roc_auc_score(y_test, meta_train[test_index])

# Training Model

In [6]:
meta_train = np.zeros(train.shape[0])
meta_test = []

kf = StratifiedKFold(n_splits= 10, shuffle=True, random_state=47)
for fold_id, (train_index, test_index) in enumerate(kf.split(train, target)):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]

    model_lr(x_train, x_test, y_train, y_test, test, meta_train, meta_test,train_index, test_index,fold_id)

partial_test_id["TARGET"] = np.array(meta_test).T.mean(axis=1)
partial_train_id["TARGET"] = meta_train

0.5784903308060483
0.5625117078542649
0.5709456311854141
0.5744748998051881
0.5758273442184014
0.5721305571735561
0.5748001120747195
0.5646531277237066
0.5866064291594058
0.5807686945212778


/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [7]:
train_id = train_id[["SK_ID_CURR"]].merge(partial_train_id, how="left", on="SK_ID_CURR")
test_id = test_id[["SK_ID_CURR"]].merge(partial_test_id, how="left", on="SK_ID_CURR")

train_id.to_csv("csv/{}_train.csv".format(model_name), index=False)
test_id.to_csv("csv/{}_test.csv".format(model_name), index=False)